In [ ]:
from google.colab import drive
drive.mount('/content/drive')
MODEL_SAVE_PATH = "/content/drive/MyDrive/BioBERT-Symptom-Disease"

In [ ]:
import json
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

dataset = load_dataset("duxprajapati/symptom-disease-dataset")


Opening and Reading Training data


In [ ]:
tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

Opening and reading validation data

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)

# Apply tokenization
train_dataset = dataset['train'].map(tokenize_function, batched=True)
val_dataset = dataset['test'].map(tokenize_function, batched=True)

train_labels = train_dataset["label"]
val_labels = val_dataset["label"]

model = BertForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=len(disease_mapping))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##Preprocessing

from sklearn.metrics import accuracy_score
Segregating the training data sentence-wise

In [ ]:
from sklearn.metrics import accuracy_score


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=80,
    per_device_eval_batch_size=80,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# ✅ Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# ✅ Train the model
trainer.train()

# ✅ Save the trained model to Google Drive
trainer.save_model(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)
print(f"✅ Model saved to {MODEL_SAVE_PATH}")

def predict_disease(symptoms):
    inputs = tokenizer(symptoms, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()

    return predicted_label

# ✅ Example usage: Predict disease from symptoms
symptoms_input = "fever, cough, sore throat"
predicted_label = predict_disease(symptoms_input)
print(f"Predicted Label (Disease ID): {predicted_label}")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-24-32d442352697>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.173700,2.069988
2,2.013100,1.878748
3,1.593100,1.630171
4,1.246000,1.471339
5,1.229000,1.382625


✅ Model saved to /content/drive/MyDrive/BioBERT-Symptom-Disease


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
from google.colab import drive
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import json


# ✅ Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load the saved model and tokenizer from Google Drive
model_path = "/content/drive/MyDrive/BioBERT-Symptom-Disease"  # Replace with your model path in Drive
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)  # Move model to GPU or CPU

tokenizer = BertTokenizer.from_pretrained(model_path)

# ✅ Function to predict disease from symptoms
def predict_disease(symptoms):
    # Tokenize the input symptoms
    inputs = tokenizer(symptoms, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Move the input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get the model's output
    outputs = model(**inputs)

    # Get the predicted label (disease ID)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()

    return predicted_label

# ✅ Example usage: Predict disease from symptoms
symptoms_input = "I have abdominal pain or discomfort that is worse when I eat or drink. I have a loss of appetite and feel fatigued after going to the bathroom. I feel very tired all the time"  # Change this to any symptom input you want
predicted_label = predict_disease(symptoms_input)

print(f"Predicted Label (Disease ID): {predicted_label}")

# ✅ If you have the disease mapping, you can convert the label to the disease name
with open("/content/drive/MyDrive/mapping.json", "r") as file:  # Path to your mapping.json
    disease_mapping = json.load(file)

# Reverse the mapping: numeric label -> disease name
reverse_disease_mapping = {v: k for k, v in disease_mapping.items()}

# Get the disease name
predicted_disease = reverse_disease_mapping.get(predicted_label, "Unknown Disease")
print(f"Predicted Disease: {predicted_disease}")


Predicted Label (Disease ID): 736
Predicted Disease: Peptic Ulcer Disease
